In [1]:
import sys
import os

from datetime import datetime
now = datetime.now()
date_time = now.strftime("%m%d%Y")
print("date and time:",date_time)

from scipy.optimize import curve_fit

def gauss(x,mu,sigma,A):
    norm = A/(np.sqrt(2*np.pi)*sigma)
    exp  = np.exp(-((x-mu)**2)/(2*sigma*sigma))
    return norm * exp

import uproot
import matplotlib.pylab as pylab
import numpy as np
import math
from sklearn.model_selection import train_test_split
import pickle
from scipy.interpolate import CubicSpline
import awkward
import pandas as pd
import matplotlib.colors as colors
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from scipy.interpolate import interp1d

date and time: 02082025


In [2]:
# Copied functions from ELOSS

Z = 18
A = 39.948   # g / mol
I = 188.0*(10**(-6)) # MeV
K = 0.307 # MeV * cm^2 / mol
Mmu = 105.658 # MeV for muon
Me  = 0.51 # MeV for electron
rho = 1.396 # g/cm3

def beta(gamma):
    return np.sqrt(1-(1./(gamma**2)))

def gamma(KE,mass):
    return (KE/mass)+1

beta = np.vectorize(beta)
gamma = np.vectorize(gamma)

def Wmax (KE,mass):
    g = gamma(KE,mass)
    b = beta(g)
    num = 2*Me*((b*g)**2)
    den = 1 + 2*g*Me/mass + (Me/mass)**2
    return num/den

def density(bg):

    # constants and variable names obtained from :
    # PDG elos muons table [http://pdg.lbl.gov/2016/AtomicNuclearProperties/MUE/muE_liquid_argon.pdf]
    
    C  = -5.2146
    X0 = 0.2
    X1 = 3.0
    a  = 0.19559
    m  = 3.0
    N    = 2 * np.log(10)
    
    x = np.log10(bg)
    
    if (x < X0):
        return 0.
    if (x > X1):
        return N * x + C
    addition = a*((X1-x)**m)
    return N * x + C + addition

density = np.vectorize(density)
    
# KE in MeV
# x in cm
# mass in MeV
def dpdx(KE,x,mass,I=I):
    g = gamma(KE,mass)
    b = beta(g)
    epsilon = (K/2.)*(Z/A)*(x*rho/(b*b))
    A0 = (2*Me*(b*g)**2)/I
    A1 = epsilon/I
    return (1./x) * epsilon * (np.log(A0) + np.log(A1) + 0.2 - (b*b) - density(b*g))

# in MeV/cm
def dedx(KE,mass,I=I,dens=True):
    g = gamma(KE,mass)
    b = beta(g)
    F = K * (Z/A)*(1/b)**2
    wmax = Wmax(KE,mass)
    a0 = 0.5*np.log( 2*Me*(b*g)**2 * wmax / (I*I) )
    ret = a0 - b*b
    if (dens == True):
        ret -= density(b*g)/2.
    return F * ret

dedx = np.vectorize(dedx)

def KE_from_Range(R,mass,stepmax):
    KE = 0.3 # MeV
    dist = 0.
    thisstep = 1e-3        
    while ((dist+thisstep) < R):
        eloss = rho * dedx(KE,mass,dens=True)
        #print 'step: %.03f dEdx at KE %.02f is %.02f. Total dist : %.03f'%(thisstep,KE,eloss,dist)
        KE += (eloss * thisstep)
        dist += thisstep
        # update step size in an efficient way
        if ((thisstep < dist/10.) and (dist/10. < stepmax)):
            thisstep = dist/10.
    return KE

MASS_M = 105.658
STEP = 0.1
PITCH = 0.3

KE_V = np.vectorize(KE_from_Range)


In [3]:
variables = ['run','evt','ntrack','longest','reco_nu_vtx_sce_x','reco_nu_vtx_sce_y',\
             'reco_nu_vtx_sce_z','trk_sce_start_x','trk_sce_start_y','trk_sce_start_z',\
            'trk_sce_end_x','trk_sce_end_y','trk_sce_end_z','trk_llr_pid_score',\
            'trk_llr_pid_u','trk_llr_pid_v','trk_llr_pid_y',\
            'dqdx_u','dqdx_v','dqdx_y','dedx_u','dedx_v','dedx_y',\
            'rr_u','rr_v','rr_y','pitch_u','pitch_v','pitch_y']

DATA = uproot.open('/Users/jimji/Software/Data_analysis/Research/Recom_code/data_bnb_mcc9.1_v08_00_00_25_reco2_C1_beam_good_reco2_5e19.root')['nuselection']['CalorimetryAnalyzer']
DF_DATA = DATA.arrays(variables, library="pd", how="zip")

In [4]:
# select only muons
DF_MUONS = DF_DATA.query('trk_llr_pid_score > 0. and longest==1 and ntrack>1 and (trk_sce_end_x > 10 and trk_sce_end_x < 240) and (trk_sce_end_y > -100 and trk_sce_end_y < 100) and (trk_sce_end_z > 30 and trk_sce_end_z < 1000)')
rr_y_v_Muons = np.array([])
dqdx_y_v_Muons = np.array([])
pid_score_M = np.array([])

for index,entry in DF_MUONS.iterrows():
    pid_score_M = np.concatenate((pid_score_M, entry['trk_llr_pid_score']), axis=None)
    rr_y_v_Muons = np.concatenate((rr_y_v_Muons,entry['rr_y']),axis=None)
    dqdx_y_v_Muons = np.concatenate((dqdx_y_v_Muons,entry['dqdx_y']),axis=None)


print(len(rr_y_v_Muons), len(dqdx_y_v_Muons),len(pid_score_M))

1739719 1739719 6502


In [ ]:
dEdx_Muons = []


KE = KE_V(rr_y_v_Muons, MASS_M, STEP)
dEdx_Muons = dedx(KE, MASS_M, I=I, dens=True)

print(len(dEdx_Muons))